In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPool2D, UpSampling2D, Concatenate, Input
from tensorflow.keras.models import Model

tf.config.list_physical_devices('GPU')

In [ ]:
np.random.permutation(10)

In [ ]:
input_shape = (128, 128, 3)

conv_args = dict(kernel_size=(3,3), padding="same", activation="relu")

in_unet = Input(input_shape)
x = Conv2D(32, **conv_args)(in_unet)
x1 = Conv2D(32, **conv_args)(x)
x = MaxPool2D()(x1)

x = Conv2D(64, **conv_args)(x)
x2 = Conv2D(64, **conv_args)(x)
x = MaxPool2D()(x2)

x = Conv2D(128, **conv_args)(x)
x3 = Conv2D(128, **conv_args)(x)
x = MaxPool2D()(x3)

x = Conv2D(256, **conv_args)(x)
x = Conv2D(256, **conv_args)(x)

x = UpSampling2D()(x)
x = Concatenate(axis=-1)([x, x3])
x = Conv2D(128, **conv_args)(x)
x = Conv2D(128, **conv_args)(x)

x = UpSampling2D()(x)
x = Concatenate(axis=-1)([x, x2])
x = Conv2D(64, **conv_args)(x)
x = Conv2D(64, **conv_args)(x)

x = UpSampling2D()(x)
x = Concatenate(axis=-1)([x, x1])
x = Conv2D(32, **conv_args)(x)
x = Conv2D(32, **conv_args)(x)

out_unet = Conv2D(1, kernel_size=(3,3), padding="same", activation="sigmoid")(x)

unet = Model(in_unet, out_unet)

In [ ]:
unet.compile(loss="binary_crossentropy")

In [ ]:
from data_generation import ImageStream

train_gen = ImageStream("mask_dataset/train")

img, mask = train_gen.next()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

seed = 1
train_img_generator = train_datagen.flow_from_directory(
        "mask_dataset/train",
        color_mode="rgb",
        target_size=(128, 128),
        batch_size=32,
        shuffle=True,
        classes=["image"],
        seed=seed)

train_mask_generator = train_datagen.flow_from_directory(
        'mask_dataset/train/',
        target_size=(128, 128),
        batch_size=32,
        shuffle=True,
        classes=["label"],
        seed=seed)

In [ ]:
img = train_img_generator.next()
mask = train_mask_generator.next()

In [ ]:
w, h = 4, 4
fig, ax = plt.subplots(h, w, figsize=(12,12))

for i in range(h):
    for j in range(w):
        ax[i,j].imshow(img[0][w*i+j])
        ax[i,j].imshow(np.ma.masked_equal(mask[0][w*i+j], 0), alpha=.5)
        ax[i,j].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
from data_generation import ImageStream

train_gen = ImageStream("test_dataset/train")
X, y = train_gen.load_dataset(1000)


In [ ]:
unet.fit(X, y, batch_size=20, epochs=5)

In [ ]:
unet = keras.models.load_model("trained_models/Unet/unet_v1.tf")

In [ ]:
image, mask = train_gen.next()
mask_pred = unet.predict(image)

In [ ]:
train_gen = ImageStream("test_dataset/test")
X_test, y_test = train_gen.load_dataset(2000)

In [ ]:
unet.evaluate()

In [ ]:
(mask_pred[w*i+j,:,:,0] > .5).sum()

In [ ]:
w, h = 4, 4
fig, ax = plt.subplots(h, w, figsize=(12,12))

for i in range(h):
    for j in range(w):
        ax[i,j].imshow(image[w*i+j])
        # ax[i,j].imshow(mask[w*i+j,:,:,0], cmap="gray")
        ax[i,j].imshow(np.ma.masked_less(mask_pred[w*i+j,:,:,0], .5), alpha=.5)
        ax[i,j].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
unet.save(r"trained_models\Unet\unet_v1.tf")

In [ ]:
seed = 1
image_datagen = ImageDataGenerator(rescale=1./255)
mask_datagen = ImageDataGenerator(rescale=1./255)

image_generator = image_datagen.flow_from_directory(
    'mask_dataset/train',
    class_mode=None,
    shuffle=False,
    classes=["image"],
    seed=seed)

mask_generator = mask_datagen.flow_from_directory(
    'mask_dataset/train',
    color_mode="grayscale",
    class_mode=None,
    shuffle=False,
    classes=["label"],
    seed=seed)

In [ ]:
image_generator.next().shape

In [ ]:
image_datagen = ImageDataGenerator(rescale=1./255)
mask_datagen = ImageDataGenerator(rescale=1./255)

image_generator = image_datagen.flow_from_directory(
    'mask_dataset/train',
    target_size=(128,128),
    class_mode=None,
    shuffle=False,
    classes=["image"],
    seed=seed)

mask_generator = mask_datagen.flow_from_directory(
    'mask_dataset/train',
    target_size=(128,128),
    color_mode="grayscale",
    class_mode=None,
    shuffle=False,
    classes=["label"],
    seed=seed)

In [ ]:
from data_generation import SegmentationDataGenerator

data_denerator = SegmentationDataGenerator("test_dataset/train", "test_dataset/train")

In [ ]:
unet.fit(
    data_denerator,
    steps_per_epoch=400,
    epochs=5)

In [ ]:

# data_gen_args = dict(featurewise_center=True,
#                      featurewise_std_normalization=True,
#                      rotation_range=90,
#                      width_shift_range=0.1,
#                      height_shift_range=0.1,
#                      zoom_range=0.2)

# image_datagen = ImageDataGenerator(**data_gen_args)
# mask_datagen = ImageDataGenerator(**data_gen_args)
# # Provide the same seed and keyword arguments to the fit and flow methods

# seed = 1
# image_datagen.fit(images, augment=True, seed=seed)
# mask_datagen.fit(masks, augment=True, seed=seed)

seed = 1
image_datagen = ImageDataGenerator(rescale=1./255)
mask_datagen = ImageDataGenerator(rescale=1./255)

image_generator = image_datagen.flow_from_directory(
    'mask_dataset/train',
    target_size=(128,128),
    color_mode="rgb",
    class_mode=None,
    shuffle=False,
    classes=["image"],
    seed=seed)

mask_generator = mask_datagen.flow_from_directory(
    'mask_dataset/train',
    target_size=(128,128),
    color_mode="grayscale",
    class_mode=None,
    shuffle=False,
    classes=["label"],
    seed=seed)



# combine generators into one which yields image and masks
train_generator = zip(train_img_generator, train_mask_generator)
unet.fit(
    train_generator,
    # steps_per_epoch=400,
    epochs=5)

In [ ]:
from tensorflow.keras.utils import Sequence

class DataGenerator(Sequence):
    'Generates data for Keras'
    
    def __init__(self, pair, class_map, batch_size=16, dim=(224,224,3), shuffle=True):
        'Initialization'
        self.dim = dim
        self.pair = pair
        self.class_map = class_map
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return len(self.pair) // self.batch_size

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [k for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.pair))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        batch_imgs = list()
        batch_labels = list()

        # Generate data
        for i in list_IDs_temp:
            # Store sample
            img = load_img(self.pair[i][0] ,target_size=self.dim)
            img = img_to_array(img)/255.
            batch_imgs.append(img)

            label = load_img(self.pair[i][1],target_size=self.dim)
            label = img_to_array(label)
            label = form_2D_label(label,self.class_map)
            label = to_categorical(label , num_classes = 32)
            batch_labels.append(label)
            
        return np.array(batch_imgs) ,np.array(batch_labels)